In [8]:
#############################################################################################################
# Extracting chemical-disease associations from the biological literature
# R214: Main Practical
# Jan Ondras (jo356), Trinity College
#############################################################################################################
# Check agreement between NE tags and matched concept types (target PubMed)
# Label mismatch count:  4261
# example: sed -n '1025017,1025037p' < pubmed-abstracts.conll
# ALSO CHECKED consistency of NE tags within the NE => consistent!
#############################################################################################################
# 2. cell: check concepts without beginning (on target PubMed)

import numpy as np
import matplotlib.pyplot as plt
import time
import glob

#########################
labelMismatchCnt = 0    #
NEtagLabels = []        #
#########################

# Dictionaries of total counts of occurrences of chemical/disease: 
# key=chemicalCode/diseaseCode, value=count 
# if chemical/disease is not in this dictionary then assume zero count
chemCnts = {} 
diseCnts = {}
# Dictionary of cooccurrences: key=chemicalCode+diseaseCode, value=count
cooc = {}     

TMP_chemCnts = {} # dictionaries as above, but temporary, for current sentence only
TMP_diseCnts = {}

tmpStr = '' # temporary string storing a current concept being parsed

########################################################################################
# HELPER FUNCTIONS 

# Abstract the given full concept name (c)
def abstract_concept(c):
    # Remove the following characters from (c)
    rmv_chars = " '-,/*+.!@#$%^&*()_={}[];\:<>?" 
    #rmv_chars = " " 
    c = c.translate(None, rmv_chars)
    c = c.translate(None, '"') # quotes are removed separately
    # Convert to lower case
    c = c.lower()
    return c

# End of concept check & update: update counts once a whole concept is parsed
def eoc():
    
    global NEtagLabel
    global labelMismatchCnt
    global NEtagLabels
    global LINE_NUM
    
    global tmpStr
    global TMP_diseCnts
    global TMP_chemCnts
    global cnt_matched
    global cnt_NOT_matched

    if tmpStr != '': # yes, temporary string is not empty
        # Simple approximate string matching
        ac = abstract_concept(tmpStr)
        # Is this abstracted concept in our grounding dictionary ?
        if ac in ground_dict:
            matched_MESH_concept = ground_dict[ac]
            mLabel = matched_MESH_concept[1]  # matched label = chemical / disease   
            mCode =  matched_MESH_concept[2]  # matched code of chemical / disease
            #print "YES: ", tmpStr, "<==>", matched_MESH_concept[0], "\t *** ", ac
            
            # CHECK LABEL AGREEMENT NE ~ CONCEPT
            if NEtagLabel != mLabel:
                print LINE_NUM, "Label mismatch: ", NEtagLabel, NEtagLabels[-1], mLabel
                labelMismatchCnt += 1
                if NEtagLabel != NEtagLabels[-1]: # just a sanity check, never happened
                    print LINE_NUM, NEtagLabel, NEtagLabels
                
            # CHECK LABEL AGREEMENT WITHIN NE
            for iii in NEtagLabels[1:]:
                if iii != NEtagLabels[0]:
                    print LINE_NUM, "Label mismatch of NE items within NE:", NEtagLabels, tmpStr
            
            # UPDATE COUNTS for this concept
            if mLabel == 'Disease':
                if mCode not in TMP_diseCnts.keys():
                    TMP_diseCnts[mCode] = 1
                else:
                    TMP_diseCnts[mCode] += 1
            elif mLabel == 'Chemical':
                if mCode not in TMP_chemCnts.keys():
                    TMP_chemCnts[mCode] = 1
                else:
                    TMP_chemCnts[mCode] += 1
            else:
                raise ValueError('Unknown concept!')

            cnt_matched += 1
        else:
            #print "NO :", tmpStr
            cnt_NOT_matched += 1
        
        tmpStr = ''
        
        NEtagLabels = []
        
# End of sentence update: update total counts based on temporary counts from this sentence, once a whole sentence is parsed
def eos():
    
    global chemCnts
    global diseCnts
    global cooc
    
    for chem in TMP_chemCnts.keys():
        # Update chemical counts
        if chem not in chemCnts.keys(): 
            chemCnts[chem] = 1
        else:
            chemCnts[chem] += 1

        for dise in TMP_diseCnts.keys():
            # Update disease counts
            if dise not in diseCnts.keys():
                diseCnts[dise] = 1
            else:
                diseCnts[dise] += 1

            # Update co-occurences - minimum of 2x in total will be considered by filtering later
            k = chem + '+' + dise # construct key into dictionary of cooccurences: chemicalCode+diseaseCode, always this ordering
            if k not in cooc.keys():
                cooc[k] = 1
            else:
                cooc[k] += 1

# END OF HELPER FUNCTIONS 
########################################################################################

# Load MESH concepts: <name, type (Chemical or Disease), code>
MESH = np.loadtxt('./../Dataset/CDR_MeSH.tsv', delimiter='\t', skiprows=0, dtype=str) # <name, label, code>

# Construct the Gounding dictionary
# maps grounded/abstracted concept to list [full concept name, type (Chemical or Disease), code]
ground_dict = {}
for r in MESH:
    ac = abstract_concept(r[0])
    ground_dict[ac] = r
    
########################################################################################
########################################################################################
                    
# Iterate over PubMed abstracts and associated tags (produced by named entity recogniser)

st = time.time()
N_sentences = 0      # to count number of sentences
LINE_NUM = 0
cnt_matched = 0      # count matched concepts
cnt_NOT_matched = 0  # count not-matched concepts
cnt_cwb = 0          # count concepts without beginning tag B-...

with open('/home/janciovec/Desktop/pubmed-abstracts.conll') as f_NE:     # named entities;  301,084,933 lines, using grep -c '' pubmed-abstracts_tags
    with open('/home/janciovec/Desktop/pubmed-abstracts_tags') as f_TA:  # associated tags; 301,084,933 lines
        
        # Iterate over all lines
        while True: 
            
            LINE_NUM += 1

            # Read line - named entity
            rl = f_NE.readline()
            rlwn = rl.split('\n')[0] # read line skipping the new line character

            # End of file
            if rl == '':
                if f_TA.readline() != '':
                    raise ValueError("File with tags does not match the file with named entities!")
                # End of concept check & update
                eoc()
                # End of sentence update
                eos()
                break

            # End of sentence
            elif rlwn == '':
                if f_TA.readline() != '\n':
                    raise ValueError("File with tags does not match the file with named entities!")
                # End of concept check & update
                eoc()
                # End of sentence update
                eos()
                # Reset temporary counters for next sentence
                TMP_chemCnts = {}
                TMP_diseCnts = {}
                N_sentences += 1

            # Beginning or within sentence
            else:
                # Read associated tag
                tag = f_TA.readline().split('\n')[0]
                # Named entity - first item at the line
                NE = rlwn.split('\t')[0]
                
                # Beginning of concept
                if tag[0] == 'B': 

                    # End of concept check & update - since there could be a concept ending right now
                    eoc()
                    # Start new concept
                    tmpStr = NE
                    NEtagLabel = tag[2:]
                    NEtagLabels.append( NEtagLabel )
                    
                # Continuation of the concept name
                elif tag[0] == 'I': 
                    
                    if tmpStr == '': # Concept without beginning: start new one from this string
                        if TMP_chemCnts != {}:
                            #print LINE_NUM, "Concept without beginning! NOT BEGIN OF SENTENCE"
                            cnt_cwb += 1
                            
#                     else:
#                         # from NE item to NE item, check if consecutive NE item tags agree
#                         if NEtagLabel != tag[2:]:
#                             print "NE item tag change within NE:", LINE_NUM, tmpStr
                    
                    tmpStr = tmpStr + NE # concatenate words without a space between them
        
                    NEtagLabel = tag[2:]
                    NEtagLabels.append( NEtagLabel )
                    
                elif tag[0] == 'O':
                    # End of concept check & update
                    eoc()
                    
                    NEtagLabel = tag[0]
#                     NEtagLabels.append( NEtagLabel )
                else:
                    raise ValueError("Unknown tag!")
                    
            if LINE_NUM % 30000000 == 0:
                print 100.*LINE_NUM/301084933, " % processed; time so far: ", time.time()-st, " s"

print "Time taken ...", time.time()-st, " s"

print "Total # concepts: ", cnt_NOT_matched + cnt_matched
print "Matched concepts: ", cnt_matched

print "Concepts without beginning tag (B-...): ", cnt_cwb

print "Total # sentences: ", N_sentences

print "Label mismatch count: ", labelMismatchCnt

##############################################################################################################
# Total time: 1568.43784904 seconds = 26.14 minutes (with printing), 20.42 min
# Total # concepts:   9,105,005
# Matched concepts:   5,940,715
# Concepts without beginning tag (B-...):  452
# Total # sentences: 10,573,978

# To give examples of concepts witout beginning
# sed -n '1750573,1750577p' < pubmed-abstracts_tags
# sed -n '1750573,1750577p' < pubmed-abstracts.conll

53462 Label mismatch:  Disease Disease Chemical
113231 Label mismatch:  Chemical Chemical Disease
211427 Label mismatch:  Chemical Chemical Disease
211539 Label mismatch:  Chemical Chemical Disease
211561 Label mismatch:  Chemical Chemical Disease
224549 Label mismatch:  Chemical Chemical Disease
332022 Label mismatch:  Chemical Chemical Disease
590789 Label mismatch:  Chemical Chemical Disease
622167 Label mismatch:  Chemical Chemical Disease
1025027 Label mismatch:  Chemical Chemical Disease
1026812 Label mismatch:  Chemical Chemical Disease
1027011 Label mismatch:  Chemical Chemical Disease
1091093 Label mismatch:  Chemical Chemical Disease
1091625 Label mismatch:  Chemical Chemical Disease
1091666 Label mismatch:  Chemical Chemical Disease
1091676 Label mismatch:  Chemical Chemical Disease
1091697 Label mismatch:  Chemical Chemical Disease
1091917 Label mismatch:  Chemical Chemical Disease
1266307 Label mismatch:  Chemical Chemical Disease
1357835 Label mismatch:  Chemical Chemical

13393092 Label mismatch:  Chemical Chemical Disease
13449879 Label mismatch:  Chemical Chemical Disease
13450039 Label mismatch:  Chemical Chemical Disease
13500433 Label mismatch:  Chemical Chemical Disease
13692340 Label mismatch:  Disease Disease Chemical
13732976 Label mismatch:  Chemical Chemical Disease
13914234 Label mismatch:  Chemical Chemical Disease
13914324 Label mismatch:  Chemical Chemical Disease
13914333 Label mismatch:  Chemical Chemical Disease
13914385 Label mismatch:  Chemical Chemical Disease
13914422 Label mismatch:  Chemical Chemical Disease
13914433 Label mismatch:  Chemical Chemical Disease
13914478 Label mismatch:  Chemical Chemical Disease
13994028 Label mismatch:  Chemical Chemical Disease
13994046 Label mismatch:  Chemical Chemical Disease
13994069 Label mismatch:  Chemical Chemical Disease
13994109 Label mismatch:  Chemical Chemical Disease
14475662 Label mismatch:  Chemical Chemical Disease
14475676 Label mismatch:  Chemical Chemical Disease
14479617 Labe

25721316 Label mismatch:  Chemical Chemical Disease
25770213 Label mismatch:  Chemical Chemical Disease
26114289 Label mismatch:  Chemical Chemical Disease
26238607 Label mismatch:  Chemical Chemical Disease
26310401 Label mismatch:  Chemical Chemical Disease
26346605 Label mismatch:  Chemical Chemical Disease
26395890 Label mismatch:  Chemical Chemical Disease
26395901 Label mismatch:  Chemical Chemical Disease
26395948 Label mismatch:  Chemical Chemical Disease
26395980 Label mismatch:  Chemical Chemical Disease
26396129 Label mismatch:  Chemical Chemical Disease
26396179 Label mismatch:  Chemical Chemical Disease
26475209 Label mismatch:  Chemical Chemical Disease
26659991 Label mismatch:  Chemical Chemical Disease
26660057 Label mismatch:  Chemical Chemical Disease
26660336 Label mismatch:  Chemical Chemical Disease
26660400 Label mismatch:  Chemical Chemical Disease
26867237 Label mismatch:  Chemical Chemical Disease
26868955 Label mismatch:  Chemical Chemical Disease
26959512 Lab

36095557 Label mismatch:  Chemical Chemical Disease
36161195 Label mismatch:  Chemical Chemical Disease
36213314 Label mismatch:  Chemical Chemical Disease
36563865 Label mismatch:  Disease Disease Chemical
36584205 Label mismatch:  Chemical Chemical Disease
36590899 Label mismatch:  Chemical Chemical Disease
36626678 Label mismatch:  Disease Disease Chemical
36640008 Label mismatch:  Chemical Chemical Disease
36748715 Label mismatch:  Chemical Chemical Disease
36755322 Label mismatch:  Chemical Chemical Disease
36789671 Label mismatch:  Chemical Chemical Disease
36883952 Label mismatch:  Chemical Chemical Disease
36900930 Label mismatch:  Chemical Chemical Disease
36967188 Label mismatch:  Disease Disease Chemical
37036796 Label mismatch:  Chemical Chemical Disease
37073139 Label mismatch:  Chemical Chemical Disease
37177603 Label mismatch:  Chemical Chemical Disease
37320891 Label mismatch:  Chemical Chemical Disease
37320912 Label mismatch:  Chemical Chemical Disease
37412074 Label 

45280671 Label mismatch:  Chemical Chemical Disease
45307113 Label mismatch:  Chemical Chemical Disease
45307169 Label mismatch:  Chemical Chemical Disease
45307412 Label mismatch:  Chemical Chemical Disease
45402496 Label mismatch:  Chemical Chemical Disease
45404055 Label mismatch:  Chemical Chemical Disease
45404082 Label mismatch:  Chemical Chemical Disease
45505289 Label mismatch:  Chemical Chemical Disease
45544323 Label mismatch:  Chemical Chemical Disease
45575273 Label mismatch:  Chemical Chemical Disease
45575426 Label mismatch:  Chemical Chemical Disease
45575586 Label mismatch:  Chemical Chemical Disease
45583027 Label mismatch:  Chemical Chemical Disease
45710157 Label mismatch:  Chemical Chemical Disease
45858341 Label mismatch:  Chemical Chemical Disease
45870931 Label mismatch:  Chemical Chemical Disease
45909122 Label mismatch:  Chemical Chemical Disease
45911694 Label mismatch:  Chemical Chemical Disease
45933624 Label mismatch:  Disease Disease Chemical
45941738 Labe

...

260381031 Label mismatch:  Chemical Chemical Disease
260381062 Label mismatch:  Chemical Chemical Disease
260381090 Label mismatch:  Chemical Chemical Disease
260381099 Label mismatch:  Chemical Chemical Disease
260381109 Label mismatch:  Chemical Chemical Disease
260381152 Label mismatch:  Chemical Chemical Disease
260381159 Label mismatch:  Chemical Chemical Disease
260381208 Label mismatch:  Chemical Chemical Disease
260430108 Label mismatch:  Chemical Chemical Disease
260434018 Label mismatch:  Chemical Chemical Disease
260470638 Label mismatch:  Chemical Chemical Disease
260470737 Label mismatch:  Chemical Chemical Disease
260481163 Label mismatch:  Chemical Chemical Disease
260481633 Label mismatch:  Chemical Chemical Disease
260497612 Label mismatch:  Chemical Chemical Disease
260497679 Label mismatch:  Chemical Chemical Disease
260581824 Label mismatch:  Chemical Chemical Disease
260585628 Label mismatch:  Chemical Chemical Disease
260596780 Label mismatch:  Chemical Chemical D

266650757 Label mismatch:  Chemical Chemical Disease
266650883 Label mismatch:  Chemical Chemical Disease
266650919 Label mismatch:  Chemical Chemical Disease
266777273 Label mismatch:  Chemical Chemical Disease
266937290 Label mismatch:  Chemical Chemical Disease
267092530 Label mismatch:  Chemical Chemical Disease
267112889 Label mismatch:  Chemical Chemical Disease
267196696 Label mismatch:  Chemical Chemical Disease
267196783 Label mismatch:  Chemical Chemical Disease
267196831 Label mismatch:  Chemical Chemical Disease
267196944 Label mismatch:  Chemical Chemical Disease
267246800 Label mismatch:  Chemical Chemical Disease
267504697 Label mismatch:  Chemical Chemical Disease
267721705 Label mismatch:  Chemical Chemical Disease
267819920 Label mismatch:  Chemical Chemical Disease
267849328 Label mismatch:  Chemical Chemical Disease
267911577 Label mismatch:  Chemical Chemical Disease
267966781 Label mismatch:  Chemical Chemical Disease
267966912 Label mismatch:  Chemical Chemical D

277176573 Label mismatch:  Chemical Chemical Disease
277179418 Label mismatch:  Chemical Chemical Disease
277300636 Label mismatch:  Chemical Chemical Disease
277364520 Label mismatch:  Chemical Chemical Disease
277364589 Label mismatch:  Chemical Chemical Disease
277508430 Label mismatch:  Chemical Chemical Disease
277816122 Label mismatch:  Chemical Chemical Disease
277885154 Label mismatch:  Chemical Chemical Disease
277907524 Label mismatch:  Chemical Chemical Disease
278094020 Label mismatch:  Chemical Chemical Disease
278097929 Label mismatch:  Chemical Chemical Disease
278218524 Label mismatch:  Chemical Chemical Disease
278238427 Label mismatch:  Chemical Chemical Disease
278516634 Label mismatch:  Chemical Chemical Disease
278516668 Label mismatch:  Chemical Chemical Disease
278516735 Label mismatch:  Chemical Chemical Disease
278516885 Label mismatch:  Chemical Chemical Disease
278516887 Label mismatch:  Chemical Chemical Disease
278611683 Label mismatch:  Chemical Chemical D

286681971 Label mismatch:  Chemical Chemical Disease
286741075 Label mismatch:  Chemical Chemical Disease
286834001 Label mismatch:  Chemical Chemical Disease
286847677 Label mismatch:  Chemical Chemical Disease
286953194 Label mismatch:  Chemical Chemical Disease
286968225 Label mismatch:  Chemical Chemical Disease
287217857 Label mismatch:  Chemical Chemical Disease
287290248 Label mismatch:  Chemical Chemical Disease
287369137 Label mismatch:  Chemical Chemical Disease
287369167 Label mismatch:  Chemical Chemical Disease
287369204 Label mismatch:  Chemical Chemical Disease
287369224 Label mismatch:  Chemical Chemical Disease
287369231 Label mismatch:  Chemical Chemical Disease
287369239 Label mismatch:  Chemical Chemical Disease
287369287 Label mismatch:  Chemical Chemical Disease
287770585 Label mismatch:  Chemical Chemical Disease
287834446 Label mismatch:  Chemical Chemical Disease
287834513 Label mismatch:  Chemical Chemical Disease
288099608 Label mismatch:  Chemical Chemical D

299029976 Label mismatch:  Chemical Chemical Disease
299030142 Label mismatch:  Chemical Chemical Disease
299200183 Label mismatch:  Chemical Chemical Disease
299282233 Label mismatch:  Chemical Chemical Disease
299396349 Label mismatch:  Chemical Chemical Disease
299396380 Label mismatch:  Chemical Chemical Disease
299414337 Label mismatch:  Chemical Chemical Disease
299603774 Label mismatch:  Chemical Chemical Disease
299603895 Label mismatch:  Chemical Chemical Disease
299604088 Label mismatch:  Chemical Chemical Disease
299614063 Label mismatch:  Chemical Chemical Disease
299614093 Label mismatch:  Chemical Chemical Disease
299614125 Label mismatch:  Chemical Chemical Disease
299614149 Label mismatch:  Chemical Chemical Disease
299639038 Label mismatch:  Chemical Chemical Disease
299685814 Label mismatch:  Chemical Chemical Disease
299710966 Label mismatch:  Chemical Chemical Disease
299750623 Label mismatch:  Chemical Chemical Disease
99.6396588201  % processed; time so far:  1413

In [10]:
#############################################################################################################
# Check concepts without beginning (on target PubMed)
# #concepts without beginning tag (B-...) = 5 if COUNTED ONLY IF THEY DO NOT APPEAR AT THE BEGINNING OF SENTENCE
# otherwise: 452 is the total #concepts without beginning tag
#############################################################################################################

import numpy as np
import matplotlib.pyplot as plt
import time
import glob

# Dictionaries of total counts of occurrences of chemical/disease: 
# key=chemicalCode/diseaseCode, value=count 
# if chemical/disease is not in this dictionary then assume zero count
chemCnts = {} 
diseCnts = {}
# Dictionary of cooccurrences: key=chemicalCode+diseaseCode, value=count
cooc = {}     

TMP_chemCnts = {} # dictionaries as above, but temporary, for current sentence only
TMP_diseCnts = {}

tmpStr = '' # temporary string storing a current concept being parsed

########################################################################################
# HELPER FUNCTIONS 

# Abstract the given full concept name (c)
def abstract_concept(c):
    # Remove the following characters from (c)
    rmv_chars = " '-,/*+.!@#$%^&*()_={}[];\:<>?" 
    #rmv_chars = " " 
    c = c.translate(None, rmv_chars)
    c = c.translate(None, '"') # quotes are removed separately
    # Convert to lower case
    c = c.lower()
    return c

# End of concept check & update: update counts once a whole concept is parsed
def eoc():
    
    global tmpStr
    global TMP_diseCnts
    global TMP_chemCnts
    global cnt_matched
    global cnt_NOT_matched

    if tmpStr != '': # yes, temporary string is not empty
        # Simple approximate string matching
        ac = abstract_concept(tmpStr)
        # Is this abstracted concept in our grounding dictionary ?
        if ac in ground_dict:
            matched_MESH_concept = ground_dict[ac]
            mLabel = matched_MESH_concept[1]  # matched label = chemical / disease   
            mCode =  matched_MESH_concept[2]  # matched code of chemical / disease
            #print "YES: ", tmpStr, "<==>", matched_MESH_concept[0], "\t *** ", ac
            
            # UPDATE COUNTS for this concept
            if mLabel == 'Disease':
                if mCode not in TMP_diseCnts.keys():
                    TMP_diseCnts[mCode] = 1
                else:
                    TMP_diseCnts[mCode] += 1
            elif mLabel == 'Chemical':
                if mCode not in TMP_chemCnts.keys():
                    TMP_chemCnts[mCode] = 1
                else:
                    TMP_chemCnts[mCode] += 1
            else:
                raise ValueError('Unknown concept!')

            cnt_matched += 1
        else:
            #print "NO :", tmpStr
            cnt_NOT_matched += 1
        
        tmpStr = ''
        
# End of sentence update: update total counts based on temporary counts from this sentence, once a whole sentence is parsed
def eos():
    
    global chemCnts
    global diseCnts
    global cooc
    
    for chem in TMP_chemCnts.keys():
        # Update chemical counts
        if chem not in chemCnts.keys(): 
            chemCnts[chem] = 1
        else:
            chemCnts[chem] += 1

        for dise in TMP_diseCnts.keys():
            # Update disease counts
            if dise not in diseCnts.keys():
                diseCnts[dise] = 1
            else:
                diseCnts[dise] += 1

            # Update co-occurences - minimum of 2x in total will be considered by filtering later
            k = chem + '+' + dise # construct key into dictionary of cooccurences: chemicalCode+diseaseCode, always this ordering
            if k not in cooc.keys():
                cooc[k] = 1
            else:
                cooc[k] += 1

# END OF HELPER FUNCTIONS 
########################################################################################

# Load MESH concepts: <name, type (Chemical or Disease), code>
MESH = np.loadtxt('./../Dataset/CDR_MeSH.tsv', delimiter='\t', skiprows=0, dtype=str) # <name, label, code>

# Construct the Gounding dictionary
# maps grounded/abstracted concept to list [full concept name, type (Chemical or Disease), code]
ground_dict = {}
for r in MESH:
    ac = abstract_concept(r[0])
    ground_dict[ac] = r
    
########################################################################################
########################################################################################
                    
# Iterate over PubMed abstracts and associated tags (produced by named entity recogniser)

st = time.time()
N_sentences = 0      # to count number of sentences
LINE_NUM = 0
cnt_matched = 0      # count matched concepts
cnt_NOT_matched = 0  # count not-matched concepts
cnt_cwb = 0          # count concepts without beginning tag B-...

with open('/home/janciovec/Desktop/pubmed-abstracts.conll') as f_NE:     # named entities;  301,084,933 lines, using grep -c '' pubmed-abstracts_tags
    with open('/home/janciovec/Desktop/pubmed-abstracts_tags') as f_TA:  # associated tags; 301,084,933 lines
        
        # Iterate over all lines
        while True: 
            
            LINE_NUM += 1

            # Read line - named entity
            rl = f_NE.readline()
            rlwn = rl.split('\n')[0] # read line skipping the new line character

            # End of file
            if rl == '':
                if f_TA.readline() != '':
                    raise ValueError("File with tags does not match the file with named entities!")
                # End of concept check & update
                eoc()
                # End of sentence update
                eos()
                break

            # End of sentence
            elif rlwn == '':
                if f_TA.readline() != '\n':
                    raise ValueError("File with tags does not match the file with named entities!")
                # End of concept check & update
                eoc()
                # End of sentence update
                eos()
                # Reset temporary counters for next sentence
                TMP_chemCnts = {}
                TMP_diseCnts = {}
                N_sentences += 1

            # Beginning or within sentence
            else:
                # Read associated tag
                tag = f_TA.readline().split('\n')[0]
                # Named entity - first item at the line
                NE = rlwn.split('\t')[0]
                
                # Beginning of concept
                if tag[0] == 'B': 
                    # End of concept check & update - since there could be a concept ending right now
                    eoc()
                    # Start new concept
                    tmpStr = NE
                    
                # Continuation of the concept name
                elif tag[0] == 'I': 
        
                    if tmpStr == '': # Concept without beginning: start new one from this string
#                         if TMP_chemCnts != {}:
#                             print LINE_NUM, "Concept without beginning! NOT BEGIN OF SENTENCE"
#                             cnt_cwb += 1
                        print LINE_NUM, "Concept without beginning! ALL CASES", tag
                        cnt_cwb += 1
                    
                    tmpStr = tmpStr + NE # concatenate words without a space between them
                    
                elif tag[0] == 'O':
                    # End of concept check & update
                    eoc()
                else:
                    raise ValueError("Unknown tag!")
                    
            if LINE_NUM % 30000000 == 0:
                print 100.*LINE_NUM/301084933, " % processed; time so far: ", time.time()-st, " s"

print "Time taken ...", time.time()-st, " s"

print "Total # concepts: ", cnt_NOT_matched + cnt_matched
print "Matched concepts: ", cnt_matched

print "Concepts without beginning tag (B-...): ", cnt_cwb

print "Total # sentences: ", N_sentences

##############################################################################################################
# Total time: 1568.43784904 seconds = 26.14 minutes (with printing), 20.42 min
# Total # concepts:   9,105,005
# Matched concepts:   5,940,715
# Concepts without beginning tag (B-...):  452
# Total # sentences: 10,573,978

# To give examples of concepts witout beginning
# sed -n '1750573,1750577p' < pubmed-abstracts_tags
# sed -n '1750573,1750577p' < pubmed-abstracts.conll

1750575 Concept without beginning! ALL CASES I-Disease
2788688 Concept without beginning! ALL CASES I-Chemical
3387645 Concept without beginning! ALL CASES I-Disease
3701988 Concept without beginning! ALL CASES I-Chemical
4262071 Concept without beginning! ALL CASES I-Disease
4430739 Concept without beginning! ALL CASES I-Disease
5522210 Concept without beginning! ALL CASES I-Disease
6175482 Concept without beginning! ALL CASES I-Chemical
6175655 Concept without beginning! ALL CASES I-Chemical
7790449 Concept without beginning! ALL CASES I-Disease
7965579 Concept without beginning! ALL CASES I-Chemical
8317729 Concept without beginning! ALL CASES I-Disease
9115904 Concept without beginning! ALL CASES I-Disease
9323695 Concept without beginning! ALL CASES I-Chemical
10325323 Concept without beginning! ALL CASES I-Chemical
10604145 Concept without beginning! ALL CASES I-Chemical
10752818 Concept without beginning! ALL CASES I-Chemical
11628326 Concept without beginning! ALL CASES I-Chemi

102794818 Concept without beginning! ALL CASES I-Chemical
102807207 Concept without beginning! ALL CASES I-Chemical
103729368 Concept without beginning! ALL CASES I-Disease
104777377 Concept without beginning! ALL CASES I-Chemical
105331266 Concept without beginning! ALL CASES I-Chemical
105648098 Concept without beginning! ALL CASES I-Disease
107101231 Concept without beginning! ALL CASES I-Disease
107300485 Concept without beginning! ALL CASES I-Disease
107472315 Concept without beginning! ALL CASES I-Chemical
107713280 Concept without beginning! ALL CASES I-Chemical
107913119 Concept without beginning! ALL CASES I-Chemical
107913156 Concept without beginning! ALL CASES I-Chemical
109648628 Concept without beginning! ALL CASES I-Disease
109648678 Concept without beginning! ALL CASES I-Disease
110369449 Concept without beginning! ALL CASES I-Chemical
110808432 Concept without beginning! ALL CASES I-Chemical
111142531 Concept without beginning! ALL CASES I-Chemical
111961888 Concept wi

187517233 Concept without beginning! ALL CASES I-Chemical
187594382 Concept without beginning! ALL CASES I-Chemical
187688064 Concept without beginning! ALL CASES I-Disease
189014163 Concept without beginning! ALL CASES I-Disease
189515511 Concept without beginning! ALL CASES I-Chemical
189609348 Concept without beginning! ALL CASES I-Disease
190004631 Concept without beginning! ALL CASES I-Disease
190887521 Concept without beginning! ALL CASES I-Chemical
192175944 Concept without beginning! ALL CASES I-Disease
192303686 Concept without beginning! ALL CASES I-Disease
192348547 Concept without beginning! ALL CASES I-Chemical
192348585 Concept without beginning! ALL CASES I-Chemical
193415809 Concept without beginning! ALL CASES I-Chemical
193753127 Concept without beginning! ALL CASES I-Disease
194037951 Concept without beginning! ALL CASES I-Disease
194399353 Concept without beginning! ALL CASES I-Chemical
195231539 Concept without beginning! ALL CASES I-Disease
195332457 Concept witho

284736929 Concept without beginning! ALL CASES I-Disease
285586130 Concept without beginning! ALL CASES I-Chemical
286071526 Concept without beginning! ALL CASES I-Disease
286196961 Concept without beginning! ALL CASES I-Chemical
286668365 Concept without beginning! ALL CASES I-Chemical
287261024 Concept without beginning! ALL CASES I-Disease
287473215 Concept without beginning! ALL CASES I-Chemical
287493237 Concept without beginning! ALL CASES I-Chemical
288367511 Concept without beginning! ALL CASES I-Disease
289733315 Concept without beginning! ALL CASES I-Chemical
290087129 Concept without beginning! ALL CASES I-Disease
291171784 Concept without beginning! ALL CASES I-Chemical
291600334 Concept without beginning! ALL CASES I-Chemical
292310442 Concept without beginning! ALL CASES I-Disease
293915053 Concept without beginning! ALL CASES I-Disease
293943886 Concept without beginning! ALL CASES I-Chemical
294582644 Concept without beginning! ALL CASES I-Chemical
294869662 Concept wit